In [1]:
import torch
import pandas as pd
import os.path as osp
import numpy as np
from ast import literal_eval
from PIL import Image

In [2]:
## Prepare the  dataset (merge it with the emotion-histograms.)
datasetname = 'COCO' #ArtEmis, Flickr30K,  VizWiz, COCO

if datasetname == 'ArtEmis':
    datafile = f'../Dataset/{datasetname}/{datasetname}_IdC/{datasetname}_IdCII_3ErrType.csv'
    img_dir = f"../Dataset/{datasetname}/{datasetname}_IdC/Images/rawImages"
    df = pd.read_csv(datafile)
    df = df[df.split=='test']
else:
    datafile = f'../Dataset/{datasetname}/{datasetname}_IdCII_3ErrType.csv'
    img_dir = f"../Dataset/{datasetname}/Images/rawImages"
    df = pd.read_csv(datafile)
df.reset_index(drop=True,inplace=True)
print('Number of caption sets in the test set:', len(df))
df.img_files = [osp.join(img_dir,imgfile) for imgfile in df.img_files]

Number of caption sets in the test set: 1699


In [3]:
df['captSet_text'] = df['captSet_text'].apply(literal_eval)
df['refCaptSet'] = df['refCaptSet'].apply(literal_eval)

In [4]:
from SMURF.smurf.eval import preprocess,smurf_eval_captions
from SMURF.smurf.system_analysis import smurf_system_analysis

In [5]:
scores = []
no_imgs =len(df)
cands = []
refs = []
for _,row in df.iterrows():
    refs_processed = [preprocess(ref) for ref in row['refCaptSet']]
    for cand in row['captSet_text']:
        cands.append(preprocess(cand))
        refs.append(refs_processed)
meta_scorer = smurf_eval_captions(refs, cands, fuse=True)
scores = meta_scorer.evaluate()

setting up scorers...
computing SPARCS score...
Mean SPARCS score: 0.097. Computed in 2.36 seconds.
computing MIMA score...
Mean MIMA score: 0.464. Computed in 10.01 seconds.
computing SPURTS score...
Mean SPURTS score: 0.426. Computed in 6.88 seconds.
computing SMURF score...
Mean SMURF score: -0.739. Computed in 0.10 seconds.


In [6]:
SMURF = np.array(scores['SMURF']).reshape([-1,len(row['captSet_text'])])
SPURTS = np.array(scores['SPURTS']).reshape([-1,len(row['captSet_text'])])
SPARCS = np.array(scores['SPARCS']).reshape([-1,len(row['captSet_text'])])

In [7]:
import numpy as np
no_errType = 3
cnt_corr_all = 0
cnt_incorr_all = 0
print("Dataset:",datasetname,", Number of caption sets:",no_imgs)
print("SMURF")
for errType in range(1,no_errType+1):
    cnt_corr = 0
    cnt_incorr = 0
    for sim in SMURF:
        if sim[0] > sim[errType]:
            cnt_corr +=1
            cnt_corr_all +=1
        else:
            cnt_incorr +=1
            cnt_incorr_all +=1
    print(f"Accuracy at errType={errType}:{cnt_corr}/{cnt_corr+cnt_incorr}=",cnt_corr/(cnt_corr+cnt_incorr))

print(f"Accuracy for all types:{cnt_corr_all}/{cnt_corr_all+cnt_incorr_all}=",cnt_corr_all/(cnt_corr_all+cnt_incorr_all))

cnt_corr_all = 0
cnt_incorr_all = 0
print("Dataset:",datasetname,", Number of caption sets:",no_imgs)
print("SPURTS")
for errType in range(1,no_errType+1):
    cnt_corr = 0
    cnt_incorr = 0
    for sim in SPURTS:
        if sim[0] > sim[errType]:
            cnt_corr +=1
            cnt_corr_all +=1
        else:
            cnt_incorr +=1
            cnt_incorr_all +=1
    print(f"Accuracy at errType={errType}:{cnt_corr}/{cnt_corr+cnt_incorr}=",cnt_corr/(cnt_corr+cnt_incorr))

print(f"Accuracy for all types:{cnt_corr_all}/{cnt_corr_all+cnt_incorr_all}=",cnt_corr_all/(cnt_corr_all+cnt_incorr_all))

cnt_corr_all = 0
cnt_incorr_all = 0
print("Dataset:",datasetname,", Number of caption sets:",no_imgs)
print("SPARCS")
for errType in range(1,no_errType+1):
    cnt_corr = 0
    cnt_incorr = 0
    for sim in SPARCS:
        if sim[0] > sim[errType]:
            cnt_corr +=1
            cnt_corr_all +=1
        else:
            cnt_incorr +=1
            cnt_incorr_all +=1
    print(f"Accuracy at errType={errType}:{cnt_corr}/{cnt_corr+cnt_incorr}=",cnt_corr/(cnt_corr+cnt_incorr))

print(f"Accuracy for all types:{cnt_corr_all}/{cnt_corr_all+cnt_incorr_all}=",cnt_corr_all/(cnt_corr_all+cnt_incorr_all))


Dataset: COCO , Number of caption sets: 1699
SMURF
Accuracy at errType=1:583/1699= 0.3431430253090053
Accuracy at errType=2:372/1699= 0.21895232489699823
Accuracy at errType=3:301/1699= 0.17716303708063566
Accuracy for all types:1256/5097= 0.24641946242887974
Dataset: COCO , Number of caption sets: 1699
SPURTS
Accuracy at errType=1:652/1699= 0.3837551500882872
Accuracy at errType=2:764/1699= 0.4496762801648028
Accuracy at errType=3:1155/1699= 0.6798116539140671
Accuracy for all types:2571/5097= 0.5044143613890524
Dataset: COCO , Number of caption sets: 1699
SPARCS
Accuracy at errType=1:333/1699= 0.19599764567392583
Accuracy at errType=2:272/1699= 0.16009417304296644
Accuracy at errType=3:197/1699= 0.11595055915244261
Accuracy for all types:802/5097= 0.1573474592897783


In [8]:
Dataset: COCO , Number of caption sets: 1699
SMURF
Accuracy at errType=1:583/1699= 0.3431430253090053
Accuracy at errType=2:372/1699= 0.21895232489699823
Accuracy at errType=3:301/1699= 0.17716303708063566
Accuracy for all types:1256/5097= 0.24641946242887974
Dataset: COCO , Number of caption sets: 1699
SPURTS
Accuracy at errType=1:652/1699= 0.3837551500882872
Accuracy at errType=2:764/1699= 0.4496762801648028
Accuracy at errType=3:1155/1699= 0.6798116539140671
Accuracy for all types:2571/5097= 0.5044143613890524
Dataset: COCO , Number of caption sets: 1699
SPARCS
Accuracy at errType=1:333/1699= 0.19599764567392583
Accuracy at errType=2:272/1699= 0.16009417304296644
Accuracy at errType=3:197/1699= 0.11595055915244261
Accuracy for all types:802/5097= 0.1573474592897783
    
Dataset: VizWiz , Number of caption sets: 1160
SMURF
Accuracy at errType=1:310/1160= 0.2672413793103448
Accuracy at errType=2:196/1160= 0.16896551724137931
Accuracy at errType=3:83/1160= 0.07155172413793104
Accuracy for all types:589/3480= 0.1692528735632184
Dataset: VizWiz , Number of caption sets: 1160
SPURTS
Accuracy at errType=1:491/1160= 0.4232758620689655
Accuracy at errType=2:562/1160= 0.4844827586206897
Accuracy at errType=3:821/1160= 0.7077586206896552
Accuracy for all types:1874/3480= 0.5385057471264367
Dataset: VizWiz , Number of caption sets: 1160
SPARCS
Accuracy at errType=1:41/1160= 0.0353448275862069
Accuracy at errType=2:24/1160= 0.020689655172413793
Accuracy at errType=3:9/1160= 0.007758620689655172
Accuracy for all types:74/3480= 0.021264367816091954
    
Dataset: Flickr30K , Number of caption sets: 595
SMURF
Accuracy at errType=1:198/595= 0.33277310924369746
Accuracy at errType=2:121/595= 0.20336134453781513
Accuracy at errType=3:58/595= 0.09747899159663866
Accuracy for all types:377/1785= 0.21120448179271709
Dataset: Flickr30K , Number of caption sets: 595
SPURTS
Accuracy at errType=1:253/595= 0.42521008403361343
Accuracy at errType=2:253/595= 0.42521008403361343
Accuracy at errType=3:424/595= 0.7126050420168067
Accuracy for all types:930/1785= 0.5210084033613446
Dataset: Flickr30K , Number of caption sets: 595
SPARCS
Accuracy at errType=1:28/595= 0.047058823529411764
Accuracy at errType=2:27/595= 0.0453781512605042
Accuracy at errType=3:15/595= 0.025210084033613446
Accuracy for all types:70/1785= 0.0392156862745098
    
Dataset: ArtEmis , Number of caption sets: 15884
SMURF
Accuracy at errType=1:8765/15884= 0.55181314530345
Accuracy at errType=2:10184/15884= 0.6411483253588517
Accuracy at errType=3:9016/15884= 0.56761521027449
Accuracy for all types:27965/47652= 0.5868588936455973
Dataset: ArtEmis , Number of caption sets: 15884
SPURTS
Accuracy at errType=1:5818/15884= 0.3662805338705616
Accuracy at errType=2:9416/15884= 0.592797783933518
Accuracy at errType=3:11336/15884= 0.7136741374968522
Accuracy for all types:26570/47652= 0.5575841517669773
Dataset: ArtEmis , Number of caption sets: 15884
SPARCS
Accuracy at errType=1:11952/15884= 0.7524553009317553
Accuracy at errType=2:10160/15884= 0.63963737093931
Accuracy at errType=3:3899/15884= 0.24546713674137496
Accuracy for all types:26011/47652= 0.54585326953748

SyntaxError: invalid syntax (<ipython-input-8-7c9cb674b986>, line 1)